In [ ]:
import numpy as np

1. Incremental Mean
$$\mu_n = \mu_{n-1} + \frac{x_n - \mu_{n-1}}{n}$$
2. Incremental Sum of Squared Differences ($M_2$)
$$M_{2,n} = \sum_{i=1}^n (x_i - \mu_n)^2$$
$$M_{2,n} = M_{2,n-1} + (x_n - \mu_{n-1})(x_n - \mu_n)$$

In [ ]:
def sample_stats_welford(arr):
    if not arr:
        # input must be list.
        return None, None
    # nd.array: if arr.size == 0

    mean = 0.0
    m2 = 0.0  # Sum of squares of differences from the current mean
    
    for count, x in enumerate(arr, 1):
        delta = x - mean
        mean += delta / count
        delta2 = x - mean
        m2 += delta * delta2

    # Sample variance (n-1), handle n < 2 case
    variance = m2 / (count - 1) if count > 1 else 0.0
    return mean, variance

In [56]:
def average_welford(arr: list):
    if not arr:
        # input must be list.
        return None, None
    # nd.array: if arr.size == 0
    mean = 0.0
    for count, x in enumerate(arr, 1):
        delta = x - mean
        mean += delta / count
    return mean

In [ ]:
def sample_stats_unoptimized(arr):
    if not arr:
        return None, None
    n = len(arr) # this is O(1)
    mean = sum(arr) / n
    
    variance = 0
    for v in arr:
        variance += (v - mean)**2
    divisor = n - 1 if n > 1 else n
    variance /= divisor
    return mean, variance

In [63]:
# Test
np.random.seed(0)
arr = np.random.normal(loc=10, scale=1, size=(30,)).tolist()
assert np.abs(np.mean(arr) - average_welford(arr)) < 1e6

In [68]:
def moving_average_raw(arr: list, k: int):
    # k: window size
    n = len(arr)
    rolling_means = []
    if k > n:
        return rolling_means
    mean = average_welford(arr[:k])
    rolling_means.append(mean)
    for i in range(k+1, n):
        mean += (arr[i] - arr[i-k-1]) / k
        rolling_means.append(mean)
    return rolling_means

In [77]:
def moving_average(arr: list[float], k: int) -> list[float]:
    n = len(arr)
    if n < k:
        return []
    return [sum(arr[i:i+k])/k for i in range(n-k+1)]

def moving_average_kernel(arr: list[float], k: int) -> list[float]:
    kernel = np.ones(k) / k
    return np.convolve(arr, kernel, mode="valid")

In [74]:
moving_average(arr, 20)

[10.569334592945633,
 10.353482484855546,
 10.366155554209204,
 10.36044046494689,
 10.211287553966496,
 10.23139738565838,
 10.207542995922262,
 10.162326500911053,
 10.160535168824648,
 10.242335072132247,
 10.295273085530345]

In [72]:
moving_average_raw(arr, 20)

[10.569334592945633,
 10.513812905419268,
 10.537026854943882,
 10.450981704718274,
 10.452424775957581,
 10.286328592720144,
 10.337480412579037,
 10.280616799201466,
 10.364823620334274,
 10.443452501418966]